Implement the Continuous Bag of Words (CBOW) Model. Stages can be: 
    a. Data preparation 
    b. Generate training data 
    c. Train model 
    d. Output

In [2]:
# Importing libraries
from keras.preprocessing import text
from keras.utils import to_categorical  # Use to_categorical instead of np_utils
from keras.preprocessing import sequence
from keras.utils import pad_sequences


In [3]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [4]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [9]:
#generating (context word, target/label word) pairs
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)

            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        # print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [10]:
#model building
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.7130284309387

Epoch: 2 	Loss: 429.1497528553009

Epoch: 3 	Loss: 425.8350763320923

Epoch: 4 	Loss: 422.73141527175903

Epoch: 5 	Loss: 420.29770970344543



In [13]:
import pandas as pd
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.030298,-0.064939,0.026745,-0.030191,0.005892,-0.019750,0.010386,-0.033524,-0.031007,-0.027977,...,-0.036959,0.044035,-0.011974,-0.064954,-0.006982,0.018166,-0.011679,0.047226,-0.036563,0.043737
networks,0.055580,-0.023949,-0.031863,0.018082,-0.020905,0.047464,-0.007860,0.062091,-0.056392,-0.004840,...,-0.046595,-0.022302,0.046819,-0.060930,0.022194,-0.036648,0.064580,0.052613,-0.019003,0.039268
neural,-0.026750,-0.035762,0.015817,0.013218,-0.038103,-0.031846,-0.016222,-0.030985,0.040545,-0.042362,...,-0.005264,-0.033645,0.015690,0.036419,0.049149,-0.011994,0.041621,0.006916,0.015235,-0.019944
and,0.015252,-0.010458,-0.042086,-0.024104,-0.007253,0.002972,0.001348,0.021416,-0.031375,0.033285,...,0.045045,0.028468,-0.040005,-0.041114,0.044015,-0.019579,0.024688,-0.010319,-0.014231,-0.025067
as,-0.038276,-0.003796,0.002514,-0.049725,-0.039741,-0.018945,0.012319,-0.021977,-0.014117,-0.049707,...,0.028397,0.009471,-0.009517,-0.028258,0.024032,-0.012097,0.016319,-0.043118,0.009870,0.025188


In [14]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['inspection', 'neural', 'also', 'board', 'surpassing']}